<a href="https://colab.research.google.com/github/Sohamcoder1/Fake-News-Detector/blob/soham-branch/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import re
import string
import requests
from requests.utils import quote

In [ ]:
#Real time news api (https://gnews.io/dashboard) -- 100 requests per day
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
#Endpoints are -> {search , top-headlines}
endpoint = 'search'
#Parameters are -> {The topic the user want to search}
question = input('Enter your question: ')
parameters = f'q={quote(question)}&lang=en&max=5'
#Example (try this link to understand the url)-> https://gnews.io/api/v4/top-headlines?q=modi&lang=en&max=5&apikey=d2b27a1e3f818564545508b6c744d24a
news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'

response = requests.get(news_api_url)
data = response.json()

if 'articles' in data:
    for article in data['articles']:
        print(f"Published Time: {article['publishedAt']}")
        print(f"Title: {article['title']}")
        print(f"Description: {article['description']}")
        print(f"Image: {article['image']}")
        print(f"Content: {article['content']}")
        print(f"Source Name: {article['source']['name']}")
        print(f"Source URL: {article['source']['url']}")
        print(f"Source Country: {article['source']['country']}")
        print(f"URL: {article['url']}\n")
else:
    print(data)

Enter your question: g20 meet
Published Time: 2025-08-21T12:06:08Z
Title: India-Russia To Strengthen Trade Ties To Offset Trump Tariffs, BIG Takeaways From Jaishankar-Lavrov Meet
Description: EAM S. Jaishankar and Russian Foreign Minister Sergey Lavrov reaffirmed India–Russia’s deep-rooted ties in Moscow, pledging cooperation in labour, trade, and connectivity. The leaders announced new consulates, backed anti-terror efforts, and vowed closer engagement in BRICS, G20, and SCO.
Image: https://img.republicworld.com/all_images/jaishankar-lavrov-meet-1755781005544-16_9.webp
Content: External Affairs Minister S. Jaishankar and his Russian counterpart Sergey Lavrov on Thursday underlined that India–Russia ties remain anchored in “trust” and “shared interests”, while opening the door to new avenues of cooperation. The leaders addre... [1929 chars]
Source Name: Republic World
Source URL: https://www.republicworld.com
Source Country: in
URL: https://www.republicworld.com/business/india-russia-t

In [ ]:
Soham